In [1]:
include("src/pkgs.jl")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


In [2]:
include("src/riot.jl")
using Main.Riot

In [ ]:
leagues = ["challenger"]

function scrape_match(id)
    mdata = load_match(id)
    mdata
end

function scrape_summoner(data)
    id = data["summonerId"]
    sdata = load_summoner(id)
    puuid = sdata["puuid"]
    matches = load_matches_for(puuid)
    map(scrape_match, matches)
end

function scrape_league(l)
    ldata = load_league(l)
    map(scrape_summoner, ldata["entries"])
end

map(scrape_league, leagues)

Loading league challenger
Loading https://na1.api.riotgames.com/tft/league/v1/challenger -> cache/get-e2df10eaebc8ae8d9bed7951d17e48b4.json
Loading summoner hJVCiY1IT52HU_RD8Ko9lhaGIDuCHAKBBP7wfHMobLM9wzo
Loading https://na1.api.riotgames.com/tft/summoner/v1/summoners/hJVCiY1IT52HU_RD8Ko9lhaGIDuCHAKBBP7wfHMobLM9wzo -> cache/get-c00dd760115ba264843c32adcf122128.json
Loading matches for yh1UhhyMCruRwsmYlNXjEh0802T5VAIZ4n5yn1r1OuEPFFfluuvq40dZygP7cMekXsLh9j46MGBHhg
Loading https://americas.api.riotgames.com/tft/match/v1/matches/by-puuid/yh1UhhyMCruRwsmYlNXjEh0802T5VAIZ4n5yn1r1OuEPFFfluuvq40dZygP7cMekXsLh9j46MGBHhg/ids -> cache/get-409f248cc03b938dd71e91671f470a9d.json
Loading match NA1_4322256054
Loading https://americas.api.riotgames.com/tft/match/v1/matches/NA1_4322256054 -> cache/match-174f480a6b722ce7407ae4d507bf3eed.json
Loading match NA1_4321136124
Loading https://americas.api.riotgames.com/tft/match/v1/matches/NA1_4321136124 -> cache/match-42b0f2f33738077fac341c99dfd0307e.json
Load